In [301]:
# Initial imports
import numpy as np
import pandas as pd
import hvplot.pandas
from collections import Counter
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from seaborn import set_style
set_style("whitegrid")

# Read the CSV and Perform Basic Data Cleaning

In [456]:
# import Final_project_NYC_BIKE_LANE_cleaned.csv data to match accident site 
file_path = "Resources/Segment2/Final_project_NYC_Bike_All_ML.csv"
ML = pd.read_csv(file_path)
print(ML.shape)
ML

(9193, 28)


,COLLISION_ID,DATES,TIME,DAY_OF_WEEK,MONTH,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN,BIKE_LANE
0,4271563,"January 2, 2020",13:00:00,Thursday,January,MANHATTAN,10003,40.714165,-74.006320,1,...,Unspecified,sedan,Bike,CHAMBERS STREET,33.94,10000.0,61,9.17,Clear,1
1,4268322,"January 2, 2020",16:00:00,Thursday,January,MANHATTAN,10012,40.725643,-73.992070,1,...,Unspecified,sedan,Bike,BOWERY,42.03,10000.0,54,9.17,Clear,1
2,4268207,"January 2, 2020",17:00:00,Thursday,January,BROOKLYN,11221,40.693874,-73.917770,1,...,Unspecified,station wagon/sport utility vehicle,Bike,CENTRAL AVENUE,44.60,10000.0,48,12.75,Rain,0
3,4268408,"January 3, 2020",8:00:00,Friday,January,BRONX,10456,40.820747,-73.906006,1,...,Unspecified,e-bike,Taxi,JACKSON AVENUE,46.78,10000.0,60,8.05,Rain,0
4,4269187,"January 3, 2020",11:00:00,Friday,January,BROOKLYN,11205,40.691017,-73.954475,1,...,Unsafe Speed,sedan,E-Bike,DE KALB AVENUE,44.92,10000.0,74,8.05,Rain,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9188,4571618,"October 9, 2022",1:00:00,Sunday,October,QUEENS,11375,40.734295,-73.850310,1,...,Unspecified,taxi,Bike,108 STREET,52.79,10000.0,53,8.05,Clear,0
9189,4571590,"October 9, 2022",6:00:00,Sunday,October,BROOKLYN,11201,40.689840,-73.978630,1,...,Unspecified,bike,Station Wagon/Sport Utility Vehicle,DE KALB AVENUE,48.74,10000.0,66,8.01,Clear,1
9190,4571393,"October 9, 2022",11:00:00,Sunday,October,BROOKLYN,11201,40.702610,-73.992516,1,...,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,bike,Bike,FRONT STREET,44.96,10000.0,74,4.61,Clear,1
9191,4571698,"October 9, 2022",12:00:00,Sunday,October,MANHATTAN,10002,40.715080,-73.992500,1,...,Unspecified,bike,Station Wagon/Sport Utility Vehicle,CANAL STREET,46.53,10000.0,72,5.75,Clear,1


In [457]:
ML.groupby(["BIKE_LANE"]).count()

,COLLISION_ID,DATES,TIME,DAY_OF_WEEK,MONTH,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN
BIKE_LANE,,,,,,,,,,,,,,,,,,,,,
0,5998,5998,5998,5998,5998,5998,5998,5998,5998,5998,...,5998,5998,5998,5998,5998,5998,5990,5998,5998,5998
1,3195,3195,3195,3195,3195,3194,3195,3195,3195,3195,...,3195,3195,3195,3195,3195,3195,3190,3195,3195,3195


In [458]:
ML.columns.tolist()

['COLLISION_ID',
 'DATES',
 'TIME',
 'DAY_OF_WEEK',
 'MONTH',
 'BOROUGH',
 'ZIP CODE',
 'LATITUDE',
 'LONGITUDE',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'CONTRIBUTING FACTOR VEHICLE 1',
 'CONTRIBUTING FACTOR VEHICLE 2',
 'VEHICLE TYPE CODE 1',
 'VEHICLE TYPE CODE 2',
 'STREET',
 'TEMP',
 'VISIBILITY',
 'HUMIDITY',
 'WIND_SPEED',
 'WEATHER_MAIN',
 'BIKE_LANE']

In [461]:
# getting columns contains strings
df_str_columns=ML.select_dtypes(include=['object'])
for col in df_str_columns.columns:
    print(col)

DATES
TIME
DAY_OF_WEEK
MONTH
BOROUGH
CONTRIBUTING FACTOR VEHICLE 1
CONTRIBUTING FACTOR VEHICLE 2
VEHICLE TYPE CODE 1
VEHICLE TYPE CODE 2
STREET
WEATHER_MAIN


In [462]:
ML["TIME"]=ML["TIME"].astype(str)

In [463]:
ML["TIME"]= ML["TIME"].map(lambda x: x.lstrip('2022-10-25'). rstrip(' ')).map(lambda x: x.lstrip('2022-10-25'). rstrip(' '))

In [464]:
ML["DATES"] =pd.to_datetime(ML["DATES"])

In [465]:
ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9193 entries, 0 to 9192
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   COLLISION_ID                   9193 non-null   int64         
 1   DATES                          9193 non-null   datetime64[ns]
 2   TIME                           9193 non-null   object        
 3   DAY_OF_WEEK                    9193 non-null   object        
 4   MONTH                          9193 non-null   object        
 5   BOROUGH                        9192 non-null   object        
 6   ZIP CODE                       9193 non-null   int64         
 7   LATITUDE                       9193 non-null   float64       
 8   LONGITUDE                      9193 non-null   float64       
 9   NUMBER OF PERSONS INJURED      9193 non-null   int64         
 10  NUMBER OF PERSONS KILLED       9193 non-null   int64         
 11  NUMBER OF PEDESTR

# Split the Data into Training and Testing

In [466]:
# getting columns contains strings
df_str_columns=ML.select_dtypes(include=['object'])
for col in df_str_columns.columns:
    print(col)

TIME
DAY_OF_WEEK
MONTH
BOROUGH
CONTRIBUTING FACTOR VEHICLE 1
CONTRIBUTING FACTOR VEHICLE 2
VEHICLE TYPE CODE 1
VEHICLE TYPE CODE 2
STREET
WEATHER_MAIN


### tarnsform dattimes and floats to integer

In [467]:
ML["TEMP"]=ML["TEMP"].astype(int)

In [468]:
ML["HUMIDITY"]=ML["HUMIDITY"].astype(int)

In [469]:
ML["WIND_SPEED"]=ML["WIND_SPEED"].astype(int)

In [470]:
ML["VISIBILITY"]=ML["VISIBILITY"].astype(np.float).astype("Int64")

/Users/dilhumarablat/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [472]:
ML.groupby(["BIKE_LANE", "NUMBER OF PERSONS KILLED"]).count()

COLLISION_ID  DATES  TIME  DAY_OF_WEEK  \
BIKE_LANE NUMBER OF PERSONS KILLED                                           
0         0                                 5997   5997  5997         5997   
          1                                    1      1     1            1   
1         0                                 3194   3194  3194         3194   
          1                                    1      1     1            1   

                                    MONTH  BOROUGH  ZIP CODE  LATITUDE  \
BIKE_LANE NUMBER OF PERSONS KILLED                                       
0         0                          5997     5997      5997      5997   
          1                             1        1         1         1   
1         0                          3194     3193      3194      3194   
          1                             1        1         1         1   

                                    LONGITUDE  NUMBER OF PERSONS INJURED  ...  \
BIKE_LANE NUMBER OF PERSONS KILLED                                        ...   
0         0                              5997                       5997  ...   
          1                                 1                          1  ...   
1         0                              3194                       3194  ...   
          1                                 1                          1  ...   

                                    CONTRIBUTING FACTOR VEHICLE 1  \
BIKE_LANE NUMBER OF PERSONS KILLED                                  
0         0                                                  5997   
          1                                                     1   
1         0                                                  3194   
          1                                                     1   

                                    CONTRIBUTING FACTOR VEHICLE 2  \
BIKE_LANE NUMBER OF PERSONS KILLED                                  
0         0                                                  5997   
          1                                                     1   
1         0                                                  3194   
          1                                                     1   

                                    VEHICLE TYPE CODE 1  VEHICLE TYPE CODE 2  \
BIKE_LANE NUMBER OF PERSONS KILLED                                             
0         0                                        5997                 5997   
          1                                           1                    1   
1         0                                        3194                 3194   
          1                                           1                    1   

                                    STREET  TEMP  VISIBILITY  HUMIDITY  \
BIKE_LANE NUMBER OF PERSONS KILLED                                       
0         0                           5997  5997        5989      5997   
          1                              1     1           1         1   
1         0                           3194  3194        3189      3194   
          1                              1     1           1         1   

                                    WIND_SPEED  WEATHER_MAIN  
BIKE_LANE NUMBER OF PERSONS KILLED                            
0         0                               5997          5997  
          1                                  1             1  
1         0                               3194          3194  
          1                                  1             1  

[4 rows x 26 columns]

In [473]:
ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9193 entries, 0 to 9192
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   COLLISION_ID                   9193 non-null   int64         
 1   DATES                          9193 non-null   datetime64[ns]
 2   TIME                           9193 non-null   object        
 3   DAY_OF_WEEK                    9193 non-null   object        
 4   MONTH                          9193 non-null   object        
 5   BOROUGH                        9192 non-null   object        
 6   ZIP CODE                       9193 non-null   int64         
 7   LATITUDE                       9193 non-null   float64       
 8   LONGITUDE                      9193 non-null   float64       
 9   NUMBER OF PERSONS INJURED      9193 non-null   int64         
 10  NUMBER OF PERSONS KILLED       9193 non-null   int64         
 11  NUMBER OF PEDESTR

In [511]:
# using lableEncoder to assign months into numerical values 
le = LabelEncoder()
ML1 = ML.copy()
ML1['MONTH'] = le.fit_transform(ML1['MONTH'])

In [512]:
# Create our features and get dummies
X = pd.get_dummies(ML1, columns= ["TIME", "VISIBILITY", "DAY_OF_WEEK","BOROUGH", 
                                 "WEATHER_MAIN", "CONTRIBUTING FACTOR VEHICLE 1"]).drop(["COLLISION_ID","LATITUDE","LONGITUDE", "STREET", "CONTRIBUTING FACTOR VEHICLE 2", "BIKE_LANE", 
                                                  "NUMBER OF PEDESTRIANS INJURED",'NUMBER OF PEDESTRIANS KILLED','NUMBER OF CYCLIST INJURED','NUMBER OF CYCLIST KILLED',
                                                   'NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED', "VEHICLE TYPE CODE 1", "VEHICLE TYPE CODE 2", "DATES"],axis=1)

# Create our target
y = ML1['BIKE_LANE']
X.head()

,MONTH,ZIP CODE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,TEMP,HUMIDITY,WIND_SPEED,TIME_3:00:00,TIME_4:00:00,TIME_6:00:00,...,CONTRIBUTING FACTOR VEHICLE 1_Shoulders Defective/Improper,CONTRIBUTING FACTOR VEHICLE 1_Steering Failure,CONTRIBUTING FACTOR VEHICLE 1_Tinted Windows,CONTRIBUTING FACTOR VEHICLE 1_Tire Failure/Inadequate,CONTRIBUTING FACTOR VEHICLE 1_Traffic Control Disregarded,CONTRIBUTING FACTOR VEHICLE 1_Turning Improperly,CONTRIBUTING FACTOR VEHICLE 1_Unsafe Lane Changing,CONTRIBUTING FACTOR VEHICLE 1_Unsafe Speed,CONTRIBUTING FACTOR VEHICLE 1_Unspecified,CONTRIBUTING FACTOR VEHICLE 1_View Obstructed/Limited
0,4,10003,1,0,33,61,9,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,10012,1,0,42,54,9,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,4,11221,1,0,44,48,12,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,10456,1,0,46,60,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,11205,1,0,44,74,8,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [513]:
X.describe()

,MONTH,ZIP CODE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,TEMP,HUMIDITY,WIND_SPEED,TIME_3:00:00,TIME_4:00:00,TIME_6:00:00,...,CONTRIBUTING FACTOR VEHICLE 1_Shoulders Defective/Improper,CONTRIBUTING FACTOR VEHICLE 1_Steering Failure,CONTRIBUTING FACTOR VEHICLE 1_Tinted Windows,CONTRIBUTING FACTOR VEHICLE 1_Tire Failure/Inadequate,CONTRIBUTING FACTOR VEHICLE 1_Traffic Control Disregarded,CONTRIBUTING FACTOR VEHICLE 1_Turning Improperly,CONTRIBUTING FACTOR VEHICLE 1_Unsafe Lane Changing,CONTRIBUTING FACTOR VEHICLE 1_Unsafe Speed,CONTRIBUTING FACTOR VEHICLE 1_Unspecified,CONTRIBUTING FACTOR VEHICLE 1_View Obstructed/Limited
count,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.00000,...,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000
mean,5.907973,10806.107691,1.041771,0.000218,64.124551,60.826172,12.162080,0.098336,0.069074,0.08996,...,0.000109,0.000218,0.000218,0.000326,0.075819,0.022735,0.009573,0.021756,0.137170,0.016861
std,3.505034,558.595380,0.242368,0.014749,16.270232,17.393948,5.534964,0.297784,0.253594,0.28614,...,0.010430,0.014749,0.014749,0.018063,0.264722,0.149065,0.097375,0.145893,0.344045,0.128756
min,0.000000,10000.000000,1.000000,0.000000,14.000000,10.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,10036.000000,1.000000,0.000000,52.000000,47.000000,8.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,11205.000000,1.000000,0.000000,67.000000,60.000000,11.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,11228.000000,1.000000,0.000000,77.000000,75.000000,15.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,11.000000,11694.000000,7.000000,1.000000,97.000000,98.000000,45.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [514]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9193 entries, 0 to 9192
Data columns (total 92 columns):
 #   Column                                                                               Non-Null Count  Dtype
---  ------                                                                               --------------  -----
 0   MONTH                                                                                9193 non-null   int64
 1   ZIP CODE                                                                             9193 non-null   int64
 2   NUMBER OF PERSONS INJURED                                                            9193 non-null   int64
 3   NUMBER OF PERSONS KILLED                                                             9193 non-null   int64
 4   TEMP                                                                                 9193 non-null   int64
 5   HUMIDITY                                                                             9193 non-null   int

In [515]:
# Check the balance of our target values
y.value_counts()

0    5998
1    3195
Name: BIKE_LANE, dtype: int64

In [516]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [517]:
# Check balances
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 4533, 1: 2361})
Counter({0: 1465, 1: 834})


# Ensemble Learners

In this section, we will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

Train the model using the training data.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score
Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [520]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
BalancedRFC_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
BalancedRFC_model= BalancedRFC_model.fit(X_train, y_train)

In [521]:
# Calculated the balanced accuracy score
y_pred = BalancedRFC_model.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7838260449660749

In [522]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1002,  463],
       [  97,  737]])

In [523]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.68      0.88      0.78      0.78      0.59      1465
          1       0.61      0.88      0.68      0.72      0.78      0.62       834

avg / total       0.80      0.76      0.81      0.76      0.78      0.60      2299



In [524]:
# List the features 
features = BalancedRFC_model.feature_importances_
features

array([5.34884969e-02, 2.46805371e-01, 5.38877143e-03, 6.10644484e-05,
       8.06351016e-02, 8.10239385e-02, 6.98344280e-02, 7.63222099e-03,
       6.74182256e-03, 7.29186605e-03, 9.12725012e-03, 9.01479476e-03,
       8.46154587e-03, 1.42031212e-02, 1.39087223e-04, 7.28388554e-05,
       2.80338827e-04, 7.10569440e-05, 2.03169566e-05, 9.24246211e-05,
       2.52222194e-04, 4.65792283e-04, 4.25541445e-04, 9.27152371e-04,
       6.93093260e-04, 9.84145457e-04, 1.15930665e-03, 3.04890719e-03,
       1.15596548e-02, 9.85317952e-03, 9.83120724e-03, 8.89360729e-03,
       1.08855619e-02, 1.10350117e-02, 1.05682864e-02, 5.52589983e-02,
       6.51226579e-02, 1.82923812e-02, 4.89362347e-02, 3.96189445e-03,
       1.31863781e-02, 1.20742319e-02, 1.95440621e-04, 2.28320452e-04,
       1.42580730e-03, 3.48178452e-03, 8.93580765e-03, 8.72924160e-04,
       8.99022015e-04, 1.11966950e-03, 1.08790117e-04, 9.81373357e-04,
       1.07885230e-03, 1.30842572e-03, 4.95584492e-04, 1.04697373e-04,
      

In [525]:
# sorted in descending order by feature importance
ranked_features = sorted(zip(features, X.columns), reverse=True)
for feature in ranked_features:
    print(f"{feature[1]}: ({feature[0]})")

ZIP CODE: (0.24680537066561697)
HUMIDITY: (0.08102393849574463)
TEMP: (0.08063510161521284)
WIND_SPEED: (0.06983442799944498)
BOROUGH_BROOKLYN: (0.0651226578586541)
BOROUGH_BRONX: (0.055258998304381676)
MONTH: (0.053488496906246456)
BOROUGH_QUEENS: (0.04893623470675259)
BOROUGH_MANHATTAN: (0.0182923812113032)
TIME_:00:00: (0.014203121241418894)
WEATHER_MAIN_Clear: (0.013186378071370734)
CONTRIBUTING FACTOR VEHICLE 1_Driver Inattention/Distraction: (0.012398223835762053)
WEATHER_MAIN_Clouds: (0.012074231900712997)
DAY_OF_WEEK_Friday: (0.011559654813446056)
DAY_OF_WEEK_Tuesday: (0.011035011723911537)
DAY_OF_WEEK_Thursday: (0.010885561875266882)
DAY_OF_WEEK_Wednesday: (0.01056828643120662)
CONTRIBUTING FACTOR VEHICLE 1_Unspecified: (0.010302802999619245)
DAY_OF_WEEK_Monday: (0.009853179524431937)
DAY_OF_WEEK_Saturday: (0.009831207236563942)
CONTRIBUTING FACTOR VEHICLE 1_Failure to Yield Right-of-Way: (0.009623337635558822)
TIME_7:00:00: (0.009127250115030636)
TIME_8:00:00: (0.009014794764

# Easy Ensemble AdaBoost Classifier

In [530]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
easyEnsemble_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
easyEnsemble_model= easyEnsemble_model.fit(X_train, y_train)


In [531]:
# Calculated the balanced accuracy score
y_pred = easyEnsemble_model.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.8023571586416873

In [532]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1044,  421],
       [  90,  744]])

In [533]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.71      0.89      0.80      0.80      0.62      1465
          1       0.64      0.89      0.71      0.74      0.80      0.65       834

avg / total       0.82      0.78      0.83      0.78      0.80      0.63      2299

